In [12]:
import findspark
findspark.init()
import pyspark
from pyspark.mllib.fpm import FPGrowth
import re
from random import random
from pyspark import SparkContext

In [13]:
import os
config = pyspark.SparkConf().set('spark.executor.memory', '2GB')\
         .set('spark.cores.max', '2')\
         .set('spark.port.maxRetries','200')\
         .set('spark.ui.port', 4040 + hash(os.getcwd().split("/")[2])%200 )    
sc = pyspark.SparkContext(appName="Professional", master='spark://polyp1:7077',conf = config )
ss = pyspark.sql.SparkSession.builder.config(conf = config).getOrCreate()

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=Professional, master=spark://polyp1:7077) created by __init__ at <ipython-input-2-ee806557b2d9>:3 

In [14]:
#### Input data
sc = ss.sparkContext
taxiRDD = sc.textFile("/scratch/ISE495/taxi_data/green_tripdata_2015-09.csv")
! head /scratch/ISE495/taxi_data/green_tripdata_2015-09.csv

In [15]:
from pyspark.sql import Row
taxiC = taxiRDD.map(lambda s: s.split(","))
taxiDF = taxiC.toDF(["VendorID","lpep_pickup_datetime","Lpep_dropoff_datetime","Store_and_fwd_flag","RateCodeID","Pickup_longitude","Pickup_latitude","Dropoff_longitude","Dropoff_latitude","Passenger_count","Trip_distance","Fare_amount","Extra","MTA_tax","Tip_amount","Tolls_amount","Ehail_fee","improvement_surcharge","Total_amount","Payment_type","Trip_type"])

print taxiDF.show()

In [18]:
taxiDF.createOrReplaceTempView("part")
taxi2DF=ss.sql("SELECT *, CONCAT(pickup_state,dropOff_state) as direction FROM (SELECT *, IF(1.323942*FLOAT(part.Pickup_longitude) + 138.669195 < FLOAT(part.Pickup_latitude),'NJ','NY') as pickup_state, IF(1.323942*FLOAT(part.Dropoff_longitude) + 138.669195 < FLOAT(part.Dropoff_latitude),'NJ','NY') as dropOff_state FROM part)")
taxi2DF.createOrReplaceTempView("2DF")
taxi2DF.show()


+--------+--------------------+---------------------+------------------+----------+-------------------+------------------+-------------------+------------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+------------+-------------+---------+
|VendorID|lpep_pickup_datetime|Lpep_dropoff_datetime|Store_and_fwd_flag|RateCodeID|   Pickup_longitude|   Pickup_latitude|  Dropoff_longitude|  Dropoff_latitude|Passenger_count|Trip_distance|Fare_amount|Extra|MTA_tax|Tip_amount|Tolls_amount|Ehail_fee|improvement_surcharge|Total_amount|Payment_type|Trip_type|pickup_state|dropOff_state|direction|
+--------+--------------------+---------------------+------------------+----------+-------------------+------------------+-------------------+------------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+-------

In [18]:
sc.stop()
ss.stop()

In [7]:
taxi2DF.groupBy("direction").count().show()


+---------+------+
|direction| count|
+---------+------+
|     NYNJ|144110|
|     NYNY|864553|
|     NJNJ|449130|
|     NJNY| 37133|
+---------+------+



In [35]:
taxi2DF.groupby('direction').agg({'Passenger_count': "mean", 'Total_amount': "mean"}).show()

+---------+------------------+--------------------+
|direction| avg(Total_amount)|avg(Passenger_count)|
+---------+------------------+--------------------+
|     NYNJ|26.448231142874565|  1.4269516341683437|
|     NYNY| 13.73546828247377|  1.4071202112536767|
|     NJNJ|12.605316634389286|  1.2877830472246343|
|     NJNY|30.270238332479874|   1.303207389653408|
+---------+------------------+--------------------+



In [15]:
from pyspark.sql.functions import to_timestamp
taxi2DF.withColumn("lpep_pickup_datetime", to_timestamp("lpep_pickup_datetime")).show()

+--------+--------------------+---------------------+------------------+----------+-------------------+------------------+-------------------+------------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+------------+-------------+---------+
|VendorID|lpep_pickup_datetime|Lpep_dropoff_datetime|Store_and_fwd_flag|RateCodeID|   Pickup_longitude|   Pickup_latitude|  Dropoff_longitude|  Dropoff_latitude|Passenger_count|Trip_distance|Fare_amount|Extra|MTA_tax|Tip_amount|Tolls_amount|Ehail_fee|improvement_surcharge|Total_amount|Payment_type|Trip_type|pickup_state|dropOff_state|direction|
+--------+--------------------+---------------------+------------------+----------+-------------------+------------------+-------------------+------------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+-------

In [21]:
k=5

In [27]:
taxi2DF.groupBy('direction').agg({'distance': 'mean','Improvement_surcharge': 'mean'}).show()

+---------+--------------------------+--------------------+
|direction|avg(Improvement_surcharge)|       avg(distance)|
+---------+--------------------------+--------------------+
|     NYNJ|       0.29185413919932673| 0.43967106196100075|
|     NYNY|       0.29369188470782304| 0.21721722123194612|
|     NJNJ|       0.29002337853150195|0.027962606418250473|
|     NJNY|       0.28107074569787216|  1.1783874461456316|
+---------+--------------------------+--------------------+

